In [ ]:
import helperfunctions as hf # helper functions written in helperfunctions.py

import os
import json
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import dit
from dit.other import tsallis_entropy
import math
import pywt
from kymatio import Scattering2D
import cv2 as cv
import medpy.io
import matplotlib.pyplot as plt
from skimage.restoration import denoise_nl_means
from pytictoc import TicToc

import multiprocessing
from joblib import Parallel, delayed

In [2]:
# input folder path
HGG_folderpath = os.path.join(os.getcwd(), "..", "BRATS2015_Training", "BRATS2015_Training", "HGG")
LGG_folderpath = os.path.join(os.getcwd(), "..", "BRATS2015_Training", "BRATS2015_Training", "LGG")
print(HGG_folderpath)
print(LGG_folderpath)

# check if the folders exist
if os.path.isdir(HGG_folderpath) and os.path.isdir(LGG_folderpath):
    print("Dataset found!")
else:
    print("ERROR: Dataset not found!")

/Users/nicklu/JHU/2022Fall/Computer Vision/final project/fall2022CV/code/../BRATS2015_Training/BRATS2015_Training/HGG
/Users/nicklu/JHU/2022Fall/Computer Vision/final project/fall2022CV/code/../BRATS2015_Training/BRATS2015_Training/LGG
Dataset found!


In [3]:
# Use the function hf.get_filepaths to get the target OT file and one type of fMRI file

# Ex. list all *.mha files in the HGG category
# Note: OT - true tumor label; the others are four different kinds of fMRI
HGG_OT_files = hf.get_filepaths(HGG_folderpath, MRtype=['OT'])
HGG_MR_Flair_files = hf.get_filepaths(HGG_folderpath, MRtype=['MR_Flair'])
HGG_MR_T1_files = hf.get_filepaths(HGG_folderpath, MRtype=['MR_T1'])
HGG_MR_T1c_files = hf.get_filepaths(HGG_folderpath, MRtype=['MR_T1c'])
HGG_MR_T2_files = hf.get_filepaths(HGG_folderpath, MRtype=['MR_T2'])

LGG_OT_files = hf.get_filepaths(LGG_folderpath, MRtype=['OT'])
LGG_MR_Flair_files = hf.get_filepaths(LGG_folderpath, MRtype=['MR_Flair'])
LGG_MR_T1_files = hf.get_filepaths(LGG_folderpath, MRtype=['MR_T1'])
LGG_MR_T1c_files = hf.get_filepaths(LGG_folderpath, MRtype=['MR_T1c'])
LGG_MR_T2_files = hf.get_filepaths(LGG_folderpath, MRtype=['MR_T2'])

print("Total number of HGG subjects:", len(HGG_OT_files))
print("Total number of LGG subjects:", len(LGG_OT_files))

Total number of HGG subjects: 220
Total number of LGG subjects: 54


In [4]:
# Assign filepaths - use OT files and one type of corresponding fMRI files at a time
OT_paths = HGG_OT_files
MR_paths = HGG_MR_Flair_files

In [7]:
# feature extraction - may take some time for each slice
print("Total number of files:", len(OT_paths))

# extract feature for each file
# for file_id in range(1):
#     print("Start processing file {}/{}".format(file_id+1, len(OT_paths)), end=' ')
#     Z = hf.get_recommended_slices_id(OT_paths, file_id)
#
#     # extract feature for each slice
#     slice_cnt = 0
#     for slice_id in range(Z, Z+1):
#         slice_cnt += 1
#         print(slice_cnt, end=' ')
#         feature = hf.extract_feature_one_slice(OT_paths, MR_paths, file_id, slice_id) # an array of features
#         print("succeeded")
#         break

def process(file_id):
    print(f"Start processing file {file_id}")
    Z = hf.get_recommended_slices_id(OT_paths, file_id)

    # extract feature for each slice
    slice_cnt = 0
    for slice_id in range(Z, Z+1):
        slice_cnt += 1
        print(slice_cnt, end=' ')
        feature = hf.extract_feature_one_slice(OT_paths, MR_paths, file_id, slice_id) # an array of features
        print("succeeded")
        return feature

num_cores = multiprocessing.cpu_count()
print(num_cores)
t = TicToc()
t.tic()
dataset = Parallel(n_jobs=num_cores)(delayed(process)(file_id) for file_id in range(1))
t.toc()

Total number of files: 220
8
Elapsed time is 32.978263 seconds.


In [ ]:
np.save('LGG_dataset.npy', dataset, allow_pickle=True)

In [ ]:
a = np.load('LGG_dataset.npy', allow_pickle=True)

In [ ]:
a.shape

In [68]:
from scipy import stats

# data is an array of size (sample size * feature dimension)
# label is an array of size (sample size, )
# Assume HGG and LGG are 0/1 labels
N = 20 # sample size
D = 100 # feature dimension
data = np.random.random((N, D))
label = [i[0] for i in np.random.randint(0, 2, (N, 1))]

In [69]:
def bonferroni_correction(pVal, alpha=0.05):
    """Perform an alpha-level test with Bonferroni correction.
    pVal: array-like
    return an array of 0 (not significant) and 1 (significant)
    """
    bonferroni_correction = alpha / len(pVal)
    result = np.zeros(len(pVal))
    result[np.where(pVal<=bonferroni_correction)] = 1
    return result

def benjamini_hochberg(pVal, fdr=0.01):
    """Perform an alpha-level test using the Benjamini Hochberg estimator.
    pVal: array-like
    return an array of 0 (not significant) and 1 (significant)
    """
    result = np.zeros(len(pVal))
    n = len(pVal)
    pVal_index = np.argsort(pVal)
    valid = np.where(pVal[pVal_index]<=np.arange(1, n+1)*fdr/n)[0]
    if len(valid) == 0:
        return result
    cutoff = valid[-1]
    result[pVal_index[:(cutoff+1)]] = 1
    return result    


columns = ['feature' + str(i) for i in range(D)]
data_df = pd.DataFrame(data, columns=columns)
y = label
corr = data_df.apply(lambda x : stats.pearsonr(x, y))
corr_pval = corr.loc[1].values
corr = corr.loc[0].values
corr_abs = np.abs(corr)
# 1 means that the feature is larger in label 1
corr_direction = [int(corr[i]>0) for i in range(len(corr))]

result = np.array([corr, corr_direction, corr_pval]).T
result_df = pd.DataFrame(result, columns=["corr", "corr_direction", "corr_pval"])
for fdr, fdr_str in [[0.05, '05'], [0.01, '01']]:
    result_df['corr_sig_bc_'+fdr_str] = bonferroni_correction(result_df['corr_pval'].values, alpha=fdr)
    result_df['corr_sig_bh_'+fdr_str] = benjamini_hochberg(result_df['corr_pval'], fdr=fdr)

result_df.head()

,corr,corr_direction,corr_pval,corr_sig_bc_05,corr_sig_bh_05,corr_sig_bc_01,corr_sig_bh_01
0,-0.223501,0.0,0.343525,0.0,0.0,0.0,0.0
1,0.019259,1.0,0.935767,0.0,0.0,0.0,0.0
2,0.425995,1.0,0.061094,0.0,0.0,0.0,0.0
3,-0.153524,0.0,0.518138,0.0,0.0,0.0,0.0
4,-0.118371,0.0,0.619161,0.0,0.0,0.0,0.0


In [70]:
# if any of corr_sig_bc_05 / corr_sig_bc_01 / corr_sig_bh_05 / corr_sig_bh_01 is 1, then that feature is significantly correlated with the label
result_df.loc[result_df['corr_sig_bc_05']==1, ]

,corr,corr_direction,corr_pval,corr_sig_bc_05,corr_sig_bh_05,corr_sig_bc_01,corr_sig_bh_01
